# Generating Formatted X-ray Maps for JoXSZ

In [2]:
from soxs.utils import soxs_cfg
soxs_cfg.set("soxs", "bkgnd_nH", "0.018") # avoid configparser error by specifying here
import soxs

In [3]:
import yt
import pyxsim

import h5py
import numpy as np
import illustris_python as il

import os
from regions import RectangleSkyRegion
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import wcs
from astropy.io import fits

## Pick a galaxy, load the data, and prepare it for yt

In [4]:
instrument = "athena_wfi"     #"lynx_lxm", "axis", "star-x", "lem_outer_array", "chandra_acisi_cy0", "athena_wfi"
area = (1.0, "m**2")

In [5]:
basePath = "../../../../sims.TNG/TNG50-1/output/"
snap = 91
z = 0.1
haloID = 166
emin = 0.7
emax = 7.0

In [6]:
halo = il.groupcat.loadSingle(basePath, snap, haloID=haloID)

In [7]:
fields = ["Coordinates", "GFM_CoolingRate", "Density", "InternalEnergy", "ElectronAbundance", "StarFormationRate", "Masses", "GFM_Metallicity", "Velocities"]

gas = il.snapshot.loadHalo(basePath, snap, haloID, 'gas')

In [8]:
header = il.groupcat.loadHeader(basePath, snap)

with h5py.File(il.snapshot.snapPath(basePath, snap),'r') as f:
    header_snap = dict(f['Header'].attrs)

In [9]:
filename = "halo_%d.hdf5" % haloID
with h5py.File(filename,'w') as f:
    for key in gas.keys():
        f['PartType0/' + key] = gas[key]
        
    # some metadata that yt demands
    f.create_group('Header')
    f['Header'].attrs['NumFilesPerSnapshot'] = 1
    f['Header'].attrs['MassTable'] = header_snap['MassTable']
    f['Header'].attrs['BoxSize'] = header['BoxSize']
    f['Header'].attrs['Time'] = header['Time']
    f['Header'].attrs['NumPart_ThisFile'] = np.array([gas['count'],0,0,0,0,0])
    f['Header'].attrs['Redshift'] = header['Redshift']
    
    
    # Must have the next six for correct units
    f["Header"].attrs["HubbleParam"] = header["HubbleParam"]
    f["Header"].attrs["Omega0"] = header["Omega0"]
    f["Header"].attrs["OmegaLambda"] = header["OmegaLambda"]

    # These correspond to the values from the TNG simulations
    f["Header"].attrs["UnitLength_in_cm"] = header_snap['UnitLength_in_cm']
    f["Header"].attrs["UnitMass_in_g"] = header_snap['UnitMass_in_g']
    f["Header"].attrs["UnitVelocity_in_cm_per_s"] = header_snap['UnitVelocity_in_cm_per_s']

## Load the data into yt

In [10]:
ds = yt.load(filename)

yt : [INFO     ] 2025-08-11 06:55:32,530 Calculating time from 9.096e-01 to be 3.934e+17 seconds
yt : [INFO     ] 2025-08-11 06:55:32,675 Parameters: current_time              = 3.934088884834162e+17 s
yt : [INFO     ] 2025-08-11 06:55:32,684 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-08-11 06:55:32,685 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-08-11 06:55:32,686 Parameters: domain_right_edge         = [35000. 35000. 35000.]
yt : [INFO     ] 2025-08-11 06:55:32,688 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-08-11 06:55:32,689 Parameters: current_redshift          = 0.09940180263022191
yt : [INFO     ] 2025-08-11 06:55:32,689 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-08-11 06:55:32,690 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-08-11 06:55:32,690 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-08-11 06:55:32,691 Parameters: hubble_const

In [11]:
c = ds.arr([halo["GroupPos"][0], halo["GroupPos"][1], halo["GroupPos"][2]], "code_length")

In [12]:
del header, gas, halo

In [13]:
def hot_gas(pfilter, data):
    pfilter1 = data[pfilter.filtered_type, "temperature"] > 3.0e5
    pfilter2 = data["PartType0", "StarFormationRate"] == 0.0
    pfilter3 = data["PartType0", "GFM_CoolingRate"] < 0.0
    return (pfilter1 & pfilter2) & pfilter3

yt.add_particle_filter("hot_gas", function=hot_gas,
                       filtered_type='gas', requires=["temperature","density"])

In [14]:
ds.add_particle_filter("hot_gas")

yt : [INFO     ] 2025-08-11 06:55:32,865 Allocating for 1.562e+06 particles
Initializing refined index: 100%|██████████| 6/6 [00:05<00:00,  1.09it/s]


True

In [15]:
nbins = 10000
source_model = pyxsim.CIESourceModel(
    "apec", emin, emax, nbins, ("hot_gas","metallicity"),
    temperature_field=("hot_gas","temperature"),
    emission_measure_field=("hot_gas", "emission_measure"),
)

pyxsim : [INFO     ] 2025-08-11 06:55:42,932 kT_min = 0.025 keV
pyxsim : [INFO     ] 2025-08-11 06:55:42,934 kT_max = 64 keV


## X-ray Emissivity and Intensity Plot

## Generate the mock x-ray emission

In [16]:
exp_time = (1e6, "s") # exposure time
redshift = z

In [17]:
width = ds.quan(1.0, "Mpc")
le = c - 0.5*width
re = c + 0.5*width
box = ds.box(le, re)

In [18]:
n_photons, n_cells = pyxsim.make_photons(f"halo_{haloID}_photons", box, redshift, area, exp_time, source_model)

pyxsim : [INFO     ] 2025-08-11 06:55:43,010 Cosmology: h = 0.6774, omega_matter = 0.3089, omega_lambda = 0.6911
pyxsim : [INFO     ] 2025-08-11 06:55:43,013 Using emission measure field '('hot_gas', 'emission_measure')'.
pyxsim : [INFO     ] 2025-08-11 06:55:43,014 Using temperature field '('hot_gas', 'temperature')'.


Preparing spectrum table :   0%|          | 0/138 [00:00<?, ?it/s]

Processing cells/particles :   0%|          | 0/963078 [00:00<?, ?it/s]

pyxsim : [INFO     ] 2025-08-11 06:59:51,452 Finished generating photons.
pyxsim : [INFO     ] 2025-08-11 06:59:51,454 Number of photons generated: 122117
pyxsim : [INFO     ] 2025-08-11 06:59:51,455 Number of cells with photons: 1879


In [19]:
n_events = pyxsim.project_photons(f"halo_{haloID}_photons", f"halo_{haloID}_events", "z", (45.,30.),
                                  absorb_model="wabs", nH=0.01)

pyxsim : [INFO     ] 2025-08-11 06:59:51,473 Foreground galactic absorption: using the wabs model and nH = 0.01.


Projecting photons from cells/particles :   0%|          | 0/1879 [00:00<?, ?it/s]

pyxsim : [INFO     ] 2025-08-11 06:59:52,175 Detected 117953 events.


In [20]:
events = pyxsim.EventList(f"halo_{haloID}_events.h5")
events.write_to_simput(f"halo_{haloID}", overwrite=True)

soxs : [INFO     ] 2025-08-11 06:59:52,319 Writing source 'halo_166' to halo_166_phlist.fits.


## Create a synthetic observation of these x-rays with Instrument

In [21]:
key = "full"
soxs.instrument_simulator(f"halo_{haloID}_simput.fits", f"halo_{haloID}_evt_{instrument}_{emin}_{emax}.fits", (10**5, "s"), instrument, (45.,30.), overwrite=True, instr_bkgnd=True, foreground=False, ptsrc_bkgnd=False)

soxs : [INFO     ] 2025-08-11 06:59:52,345 Making observation of source in halo_166_evt_athena_wfi_0.7_7.0.fits.
soxs : [INFO     ] 2025-08-11 06:59:52,797 Detecting events from source halo_166.
soxs : [INFO     ] 2025-08-11 06:59:52,799 Applying energy-dependent effective area from athena_sixte_wfi_wo_filter_v20190122.arf.
soxs : [INFO     ] 2025-08-11 06:59:52,811 Pixeling events.
soxs : [INFO     ] 2025-08-11 06:59:52,823 Scattering events with a multi_image-based PSF.
soxs : [INFO     ] 2025-08-11 06:59:52,978 15049 events were detected from the source.
soxs : [INFO     ] 2025-08-11 06:59:52,982 Scattering energies with RMF athena_wfi_sixte_v20150504.rmf.


Scattering energies :   0%|          | 0/15049 [00:00<?, ?it/s]

soxs : [INFO     ] 2025-08-11 06:59:53,105 Adding background events.
soxs : [INFO     ] 2025-08-11 06:59:53,176 Adding in instrumental background.
soxs : [INFO     ] 2025-08-11 06:59:54,293 Making 1370975 events from the instrumental background.
soxs : [INFO     ] 2025-08-11 06:59:54,467 Writing events to file halo_166_evt_athena_wfi_0.7_7.0.fits.
soxs : [INFO     ] 2025-08-11 06:59:55,458 Observation complete.


# Generating Background Files

In [22]:
from soxs.instrument import make_background_file

haloID = 166
make_background_file(
    out_file=f"halo_{haloID}_bkg_evt_full_{instrument}.fits",
    exp_time=(1e5, "s"),  # Match exposure used in science simulation
    instrument=instrument,
    sky_center=(45.0, 30.0),
    overwrite=True,
    foreground=False,        # Paper uses CIAO blanksky only (instrumental)
    instr_bkgnd=True,        # ✅ Required
    ptsrc_bkgnd=False       # No point source background
)


soxs : [INFO     ] 2025-08-11 06:59:55,566 Adding in instrumental background.
soxs : [INFO     ] 2025-08-11 06:59:55,836 Making 1369268 events from the instrumental background.
soxs : [INFO     ] 2025-08-11 06:59:56,060 Writing events to file halo_166_bkg_evt_full_athena_wfi.fits.


In [23]:
os.remove(f"halo_{haloID}_events.h5")
os.remove(f"halo_{haloID}_photons.h5")
os.remove(f"halo_{haloID}_simput.fits")
os.remove(f"halo_{haloID}_phlist.fits")
os.remove(f"halo_{haloID}.hdf5")
os.remove(f"halo_{haloID}.hdf5.ewah")